In [ ]:
import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import pickle

from DeepFMKit.experiments import Experiment
from DeepFMKit import physics

def create_intrinsic_bias_configs(params: dict) -> dict:
    """
    Configures the physics for the W-DFMI intrinsic bias landscape.
    
    This factory ensures the "golden rule" of W-DFMI testing is followed:
    for a given `m_main` and `m_witness`, it calculates the required laser `df`
    and then adjusts the witness interferometer's physical path length (OPD)
    to achieve the target `m_witness`. It also locks the witness to quadrature.
    """
    m_main = params['m_main']
    m_witness = params['m_witness']
    
    laser_config = physics.LaserConfig()
    main_ifo_config = physics.InterferometerConfig()
    witness_ifo_config = physics.InterferometerConfig()

    # No distortion for this test
    laser_config.df_2nd_harmonic_frac = 0.0

    # Calculate laser df for the target m_main
    opd_main = main_ifo_config.meas_arml - main_ifo_config.ref_arml
    if opd_main == 0: opd_main = 0.2
    laser_config.df = (m_main * sc.c) / (2 * np.pi * opd_main)

    # Design the witness IFO to achieve the target m_witness
    if laser_config.df > 0:
        opd_witness = (m_witness * sc.c) / (2 * np.pi * laser_config.df)
        witness_ifo_config.ref_arml = 0.01
        witness_ifo_config.meas_arml = witness_ifo_config.ref_arml + opd_witness
        f0 = sc.c / laser_config.wavelength
        static_fringe_phase = (2 * np.pi * f0 * opd_witness) / sc.c
        witness_ifo_config.phi = (np.pi / 2.0) - static_fringe_phase
        
    return {
        'laser_config': laser_config,
        'main_ifo_config': main_ifo_config,
        'witness_ifo_config': witness_ifo_config
    }

In [ ]:
# --- Main execution block ---
if __name__ == "__main__":
    # --- 1. Declaratively Define the Experiment ---
    fitter_to_test = 'wdfmi_ortho' # Easily switch fitters here
    
    exp = Experiment(description=f"W-DFMI Residual Bias, Fitter: {fitter_to_test}")

    # Define the 2D parameter grid
    exp.add_axis('m_main', np.linspace(3, 25, 50))
    exp.add_axis('m_witness', np.linspace(0.1, 1.0, 50))
    
    # This experiment is deterministic
    exp.n_trials = 1

    # Set the function that creates the physics configs for each grid point
    exp.set_config_factory(create_intrinsic_bias_configs)
    
    # Define the single analysis to run
    exp.add_analysis(name='wdfmi_fit', fitter_method=fitter_to_test)
    
    # --- 2. Run the Experiment ---
    results = exp.run()

    # --- 3. Save Results ---
    results_filename = f'interaction_results_{fitter_to_test}.pkl'
    if results:
        # Add metadata to the results dictionary for the plotting function
        results['fitter_name'] = fitter_to_test
        with open(results_filename, 'wb') as f:
            pickle.dump(results, f)
        print(f"Results saved to {results_filename}")

In [ ]:
# --- 4. Load and Plot the Results ---
fitter_to_test = 'wdfmi_ortho' # Ensure this matches the run you want to plot
results_filename = f'interaction_results_{fitter_to_test}.pkl'

try:
    with open(results_filename, 'rb') as f:
        loaded_results = pickle.load(f)
except FileNotFoundError:
    print(f"Results file not found: {results_filename}. Please run the experiment cell first.")
    loaded_results = None

if loaded_results:
    m_main_range = loaded_results['axes']['m_main']
    m_witness_range = loaded_results['axes']['m_witness']
    fitter_name = loaded_results['fitter_name']

    # --- THE FIX IS HERE ---
    
    # 1. Get the raw m_fit grid. Its shape is (len(m_main), len(m_witness)).
    m_fit_grid = loaded_results['wdfmi_fit']['m']['mean']

    # 2. Reshape m_main_range to (len(m_main), 1) for correct broadcasting.
    # This ensures each column of the m_fit_grid has the correct m_true subtracted from it.
    m_true_grid = m_main_range[:, np.newaxis]
    
    # 3. Calculate the bias grid. The shape is still (len(m_main), len(m_witness)).
    bias_grid = m_fit_grid - m_true_grid

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(12, 9))
    max_abs_bias = 1.0 # Use a sensitive color scale
    norm = Normalize(vmin=0, vmax=max_abs_bias)

    # 4. Transpose the final bias_grid (.T) so its shape becomes 
    # (len(m_witness), len(m_main)), which matches the Y and X axes for pcolormesh.
    im = ax.pcolormesh(
        m_main_range, m_witness_range, np.abs(bias_grid).T,
        norm=norm, cmap='coolwarm', shading='auto'
    )
    
    ax.set_title(f'W-DFMI Residual Bias, Fitter: {fitter_name}', fontsize=16)
    ax.set_xlabel(r'Main Modulation Depth, $m_{\rm main}$', fontsize=14)
    ax.set_ylabel(r'Witness Modulation Depth, $m_{\rm witness}$', fontsize=14)
    
    cbar = fig.colorbar(im, ax=ax, label=r'Absolute Residual Bias, $|\delta m|$ (rad)')
    
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()